In [1]:
import pandas as pd

from pyspark.sql import SparkSession

from pyspark.sql import SQLContext

from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS

from pyspark.sql.functions import udf, col, when, split, regexp_extract

import numpy as np

from IPython.display import Image

from IPython.display import display

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
spark = SparkSession.builder.appName("ALS_Recommend").getOrCreate()

23/11/20 19:40:55 WARN Utils: Your hostname, nhom6nth-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.59.139 instead (on interface ens33)
23/11/20 19:40:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 19:40:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/20 19:40:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Tạo một đối tượng SparkContext (sc) để quản lý các tác vụ trên Spark.
sc = spark.sparkContext

# Tạo một đối tượng SQLContext (sqlContext) để thực hiện các truy vấn SQL trên Spark DataFrame.
sqlContext = SQLContext(sc)


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
df_movies = spark.read.csv('movies.csv', inferSchema=True, header=True) 
df_movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [5]:
df_movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [6]:
df_ratings = spark.read.csv('ratings.csv', inferSchema=True, header=True) 
df_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
df_ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [8]:
df_links = spark.read.csv('links.csv', inferSchema=True, header=True) 
df_links.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- imdbId: integer (nullable = true)
 |-- tmdbId: integer (nullable = true)



In [9]:
df_links.show()

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
|      6|113277|   949|
|      7|114319| 11860|
|      8|112302| 45325|
|      9|114576|  9091|
|     10|113189|   710|
|     11|112346|  9087|
|     12|112896| 12110|
|     13|112453| 21032|
|     14|113987| 10858|
|     15|112760|  1408|
|     16|112641|   524|
|     17|114388|  4584|
|     18|113101|     5|
|     19|112281|  9273|
|     20|113845| 11517|
+-------+------+------+
only showing top 20 rows



In [10]:
# Kết hợp spark_df_movies và spark_df_ratings theo cột 'movieId'
df_kethop = df_movies.join(df_ratings, on='movieId')

# Kết hợp spark_df_kethop và spark_df_links theo cột 'movieId'
df_kethop = df_kethop.join(df_links, on='movieId')

# Hiển thị PySpark DataFrame sau khi kết hợp
df_kethop.show()

+-------+--------------------+--------------------+------+------+---------+------+------+
|movieId|               title|              genres|userId|rating|timestamp|imdbId|tmdbId|
+-------+--------------------+--------------------+------+------+---------+------+------+
|      1|    Toy Story (1995)|Adventure|Animati...|     1|   4.0|964982703|114709|   862|
|      3|Grumpier Old Men ...|      Comedy|Romance|     1|   4.0|964981247|113228| 15602|
|      6|         Heat (1995)|Action|Crime|Thri...|     1|   4.0|964982224|113277|   949|
|     47|Seven (a.k.a. Se7...|    Mystery|Thriller|     1|   5.0|964983815|114369|   807|
|     50|Usual Suspects, T...|Crime|Mystery|Thr...|     1|   5.0|964982931|114814|   629|
|     70|From Dusk Till Da...|Action|Comedy|Hor...|     1|   3.0|964982400|116367|   755|
|    101|Bottle Rocket (1996)|Adventure|Comedy|...|     1|   5.0|964980868|115734| 13685|
|    110|   Braveheart (1995)|    Action|Drama|War|     1|   4.0|964982176|112573|   197|
|    151| 

In [11]:
# Xóa các dòng có giá trị "(no genres listed)" theo số movieId
df_cleaned = df_kethop.filter(col('genres') != '(no genres listed)')

# Giữ lại một dòng duy nhất cho mỗi số movieId
df_cleaned = df_cleaned.dropDuplicates(['movieId'])

# Hiển thị PySpark DataFrame sau khi xóa
df_cleaned.show()

+-------+--------------------+--------------------+------+------+----------+------+------+
|movieId|               title|              genres|userId|rating| timestamp|imdbId|tmdbId|
+-------+--------------------+--------------------+------+------+----------+------+------+
|      1|    Toy Story (1995)|Adventure|Animati...|     1|   4.0| 964982703|114709|   862|
|      2|      Jumanji (1995)|Adventure|Childre...|     6|   4.0| 845553522|113497|  8844|
|      3|Grumpier Old Men ...|      Comedy|Romance|     1|   4.0| 964981247|113228| 15602|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|     6|   3.0| 845554349|114885| 31357|
|      5|Father of the Bri...|              Comedy|     6|   5.0| 845553938|113041| 11862|
|      6|         Heat (1995)|Action|Crime|Thri...|     1|   4.0| 964982224|113277|   949|
|      7|      Sabrina (1995)|      Comedy|Romance|     6|   4.0| 845554264|114319| 11860|
|      8| Tom and Huck (1995)|  Adventure|Children|     6|   3.0| 845555281|112302| 45325|

In [12]:
df_moviesclean = df_cleaned[['movieId', 'title', 'genres']].drop_duplicates()
df_ratingsclean = df_cleaned[['userId', 'movieId', 'rating', 'timestamp']]
df_linksclean = df_cleaned[['movieId', 'imdbId', 'tmdbId']]

In [13]:
df_moviesclean.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [14]:
# Define a regular expression to extract the year
year_pattern = r"\((\d{4})\)"

# Extract the year from the 'title' column
df_moviesclean = df_moviesclean.withColumn("year", regexp_extract(df_moviesclean["title"], year_pattern, 1))

# Split the 'title' column into 'title' and 'year' columns
df_moviesclean = df_moviesclean.withColumn("title", regexp_extract(df_moviesclean["title"], "^(.*?)\s\(\d{4}\)", 1))

# Show the updated DataFrame
df_moviesclean.show(truncate=False)

+-------+------------------------------+-------------------------------------------+----+
|movieId|title                         |genres                                     |year|
+-------+------------------------------+-------------------------------------------+----+
|1      |Toy Story                     |Adventure|Animation|Children|Comedy|Fantasy|1995|
|2      |Jumanji                       |Adventure|Children|Fantasy                 |1995|
|3      |Grumpier Old Men              |Comedy|Romance                             |1995|
|4      |Waiting to Exhale             |Comedy|Drama|Romance                       |1995|
|5      |Father of the Bride Part II   |Comedy                                     |1995|
|6      |Heat                          |Action|Crime|Thriller                      |1995|
|7      |Sabrina                       |Comedy|Romance                             |1995|
|8      |Tom and Huck                  |Adventure|Children                         |1995|
|9      |S

In [15]:
# Tách cột genres thành nhiều cột
max_genres = 10  # Số lượng cột genres tối đa bạn muốn tạo

# Thêm cột mới "genres_array" chứa mảng các thể loại
df_moviesclean = df_moviesclean.withColumn("genres_array", split(col("genres"), "\\|"))

# Tạo các cột genre_1, genre_2, ..., genre_max_genres
for i in range(1, max_genres + 1):
    df_moviesclean = df_moviesclean.withColumn(f"genre_{i}", when(col("genres_array").getItem(i - 1).isNotNull(), col("genres_array").getItem(i - 1)))

# Hiển thị kết quả
df_moviesclean = df_moviesclean.select(["movieId", "title", "year"] + [f"genre_{i}" for i in range(1, max_genres + 1)])
df_moviesclean.show(truncate=False)

+-------+------------------------------+----+---------+---------+--------+-------+--------+-------+-------+-------+-------+--------+
|movieId|title                         |year|genre_1  |genre_2  |genre_3 |genre_4|genre_5 |genre_6|genre_7|genre_8|genre_9|genre_10|
+-------+------------------------------+----+---------+---------+--------+-------+--------+-------+-------+-------+-------+--------+
|1      |Toy Story                     |1995|Adventure|Animation|Children|Comedy |Fantasy |NULL   |NULL   |NULL   |NULL   |NULL    |
|2      |Jumanji                       |1995|Adventure|Children |Fantasy |NULL   |NULL    |NULL   |NULL   |NULL   |NULL   |NULL    |
|3      |Grumpier Old Men              |1995|Comedy   |Romance  |NULL    |NULL   |NULL    |NULL   |NULL   |NULL   |NULL   |NULL    |
|4      |Waiting to Exhale             |1995|Comedy   |Drama    |Romance |NULL   |NULL    |NULL   |NULL   |NULL   |NULL   |NULL    |
|5      |Father of the Bride Part II   |1995|Comedy   |NULL     |NULL

In [17]:
# Thay thế giá trị null bằng chuỗi trống
df_moviesclean = df_moviesclean.na.fill("")

# Hiển thị kết quả
df_moviesclean.show(truncate=False)

+-------+------------------------------+----+---------+---------+--------+-------+--------+-------+-------+-------+-------+--------+
|movieId|title                         |year|genre_1  |genre_2  |genre_3 |genre_4|genre_5 |genre_6|genre_7|genre_8|genre_9|genre_10|
+-------+------------------------------+----+---------+---------+--------+-------+--------+-------+-------+-------+-------+--------+
|1      |Toy Story                     |1995|Adventure|Animation|Children|Comedy |Fantasy |       |       |       |       |        |
|2      |Jumanji                       |1995|Adventure|Children |Fantasy |       |        |       |       |       |       |        |
|3      |Grumpier Old Men              |1995|Comedy   |Romance  |        |       |        |       |       |       |       |        |
|4      |Waiting to Exhale             |1995|Comedy   |Drama    |Romance |       |        |       |       |       |       |        |
|5      |Father of the Bride Part II   |1995|Comedy   |         |    

In [18]:
df_ratingsclean.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|   1580|   3.0| 964981125|
|     1|   2366|   4.0| 964982462|
|     4|   3175|   1.0| 964538930|
|    10|   1088|   3.0|1455619275|
|    18|  32460|   3.5|1492805550|
|    18|  44022|   3.5|1455749359|
|    18|  96488|   4.0|1515532582|
|    19|   1238|   3.0| 965705784|
|    19|   1342|   2.0| 965704952|
|    19|   1591|   2.0| 965704079|
|    19|   1645|   2.0| 965711014|
|    20|   4519|   3.5|1054038247|
|    27|   2142|   3.0| 962685569|
|    32|    471|   3.0| 856737165|
|    34|   3997|   2.0|1162050228|
|    44|    833|   2.0| 869252237|
|    51|   3918|   5.0|1230931003|
|    64|   7982|   4.5|1161530147|
|    70|   1959|   4.5|1355184760|
|    73|  68135|   3.5|1464198141|
+------+-------+------+----------+
only showing top 20 rows



In [19]:
df_linksclean.show()

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|   1580| 119654|   607|
|   2366|  24216|   244|
|   3175| 177789|   926|
|   1088|  92890|    88|
|  32460| 119472|   158|
|  44022| 438097|   950|
|  96488|2125608| 84334|
|   1238|  85859| 11235|
|   1342| 103919|  9529|
|   1591| 120177| 10336|
|   1645| 118971|  1813|
|   4519|  95489| 12144|
|   2142| 101329| 10380|
|    471| 110074| 11934|
|   3997| 190374| 11849|
|    833| 116531|  9308|
|   3918|  95294|  9064|
|   7982| 365376|  4552|
|   1959|  89755|   606|
|  68135| 974661| 16996|
+-------+-------+------+
only showing top 20 rows



In [28]:
# Chia ngẫu nhiên dữ liệu trong DataFrame 'ratings_df' thành hai phần: training và validation
# Tỷ lệ chia là 80% cho training và 20% cho validation
training_df, validation_df = df_ratings.randomSplit([.8, .2])

In [29]:
# Số vòng lặp (iterations) cho thuật toán huấn luyện mô hình
iterations = 10

# Tham số regularization (chống quá mức) được sử dụng để kiểm soát độ phức tạp của mô hình và tránh overfitting
regularization_parameter = 0.1

# Rank là một tham số quan trọng liên quan đến kích thước ẩn của mô hình, thường được sử dụng trong các mô hình độ phức tạp cao như mô hình hệ thống gợi ý (recommendation systems)
rank = 4

# Một danh sách (errors) được sử dụng để lưu trữ các giá trị lỗi trong quá trình huấn luyện mô hình
errors = []

# Biến err được sử dụng để lưu trữ giá trị lỗi tạm thời trong mỗi vòng lặp
err = 0

In [30]:
# Xây dựng mô hình ALS (Alternating Least Squares) với các tham số đã được đặt trước
als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=4, userCol="userId", itemCol="movieId", ratingCol="rating")

# Huấn luyện mô hình trên tập dữ liệu huấn luyện (training_df)
model = als.fit(training_df)

# Tạo dự đoán trên tập dữ liệu validation_df sử dụng mô hình đã huấn luyện
predictions = model.transform(validation_df)

# Loại bỏ các dự đoán có giá trị NaN (không phù hợp)
new_predictions = predictions.filter(col('prediction') != np.nan)

# Sử dụng đánh giá viên (evaluator) để đánh giá chất lượng dự đoán bằng cách tính Root Mean Square Error (RMSE)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)

# In ra giá trị RMSE, một độ đo đánh giá chất lượng của mô hình hệ thống gợi ý
print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 0.8775329117435193


In [31]:
predictions.limit(10).toPandas()

,userId,movieId,rating,timestamp,prediction
0,1,6,4.0,964982224,4.730847
1,1,50,5.0,964982931,5.108908
2,1,101,5.0,964980868,4.622073
3,1,216,5.0,964981208,3.831419
4,1,223,3.0,964980985,4.631687
5,1,333,5.0,964981179,4.340956
6,1,356,4.0,964980962,5.011915
7,1,441,4.0,964980868,4.660748
8,1,457,5.0,964981909,4.838562
9,1,527,5.0,964984002,5.095624


In [39]:
predictions.join(df_moviesclean, "movieId").select("userId", "title", "year", "genre_1", "genre_2", "genre_3", "genre_4", "genre_5", "genre_6", "genre_7", "genre_8", "genre_9", "genre_10", "prediction").limit(5).toPandas()

,userId,title,year,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,prediction
0,399,Toy Story,1995,Adventure,Animation,Children,Comedy,Fantasy,,,,,,3.595572
1,275,Toy Story,1995,Adventure,Animation,Children,Comedy,Fantasy,,,,,,4.188602
2,186,Toy Story,1995,Adventure,Animation,Children,Comedy,Fantasy,,,,,,4.416960
3,46,Toy Story,1995,Adventure,Animation,Children,Comedy,Fantasy,,,,,,4.388863
4,573,Toy Story,1995,Adventure,Animation,Children,Comedy,Fantasy,,,,,,4.208315


In [40]:
for_one_user = predictions.filter(col("userId") == 599).join(df_moviesclean, "movieId").join(df_linksclean, "movieId").select("userId", "title", "year", "genre_1", "genre_2", "genre_3", "genre_4", "genre_5", "genre_6", "genre_7", "genre_8", "genre_9", "genre_10", "tmdbId", "prediction")
for_one_user.show(5)

+------+--------------------+----+-------+---------+--------+-------+-------+-------+-------+-------+-------+--------+------+----------+
|userId|               title|year|genre_1|  genre_2| genre_3|genre_4|genre_5|genre_6|genre_7|genre_8|genre_9|genre_10|tmdbId|prediction|
+------+--------------------+----+-------+---------+--------+-------+-------+-------+-------+-------+-------+--------+------+----------+
|   599|Hudsucker Proxy, The|1994| Comedy|         |        |       |       |       |       |       |       |        | 11934| 2.8471441|
|   599|       Dirty Dancing|1987|  Drama|  Musical| Romance|       |       |       |       |       |       |        |    88|  2.409865|
|   599|The Devil's Advocate|1997|  Drama|  Mystery|Thriller|       |       |       |       |       |       |        |  1813| 2.7491834|
|   599|           King Kong|1933| Action|Adventure| Fantasy| Horror|       |       |       |       |       |        |   244| 2.9146996|
|   599|  Dungeons & Dragons|2000| Action

In [41]:
import webbrowser

link = "https://www.themoviedb.org/movie/"

for movie in for_one_user.take(2):
    movieURL = link + str(movie.tmdbId)
    print(movie.title)
    webbrowser.open(movieURL)

Hudsucker Proxy, The
Dirty Dancing


In [55]:
for_one_user1 = predictions.filter(col("userId") == 10).join(df_moviesclean, "movieId").join(df_linksclean, "movieId").select("userId", "title", "year", "genre_1", "genre_2", "genre_3", "genre_4", "genre_5", "genre_6", "genre_7", "genre_8", "genre_9", "genre_10", "tmdbId", "prediction")
for_one_user1.show(5)

+------+-----------------+----+---------+---------+--------+-------+-------+-------+-------+-------+-------+--------+------+----------+
|userId|            title|year|  genre_1|  genre_2| genre_3|genre_4|genre_5|genre_6|genre_7|genre_8|genre_9|genre_10|tmdbId|prediction|
+------+-----------------+----+---------+---------+--------+-------+-------+-------+-------+-------+-------+--------+------+----------+
|    10|          Aladdin|1992|Adventure|Animation|Children| Comedy|Musical|       |       |       |       |        |   812| 3.2151449|
|    10|      Matrix, The|1999|   Action|   Sci-Fi|Thriller|       |       |       |       |       |       |        |   603| 3.2721682|
|    10|  American Beauty|1999|    Drama|  Romance|        |       |       |       |       |       |       |        |    14| 2.1927829|
|    10|      Bring It On|2000|   Comedy|         |        |       |       |       |       |       |       |        |  1588| 2.8180373|
|    10|Beautiful Mind, A|2001|    Drama|  Roman

In [56]:
import webbrowser

link = "https://www.themoviedb.org/movie/"

for movie in for_one_user1.take(2):
    movieURL = link + str(movie.tmdbId)
    print(movie.title)
    webbrowser.open(movieURL)

Aladdin
Julie & Julia


In [42]:
# Tạo ra 5 gợi ý xếp hạng cho tất cả người dùng dựa trên mô hình đã huấn luyện
userRecomments = model.recommendForAllUsers(5)

# Tạo ra 5 gợi ý xếp hạng cho tất cả sản phẩm dựa trên mô hình đã huấn luyện
movieRecomments = model.recommendForAllItems(5)

In [43]:
userRecomments.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [44]:
userRecomments.select("userId", "recommendations.movieId").show(10, False)

+------+----------------------------------+
|userId|movieId                           |
+------+----------------------------------+
|1     |[6818, 6201, 4495, 33649, 58301]  |
|2     |[51931, 60943, 59018, 62293, 7842]|
|3     |[213, 7982, 4794, 8199, 1904]     |
|4     |[158783, 89118, 25850, 4289, 7579]|
|5     |[6201, 4495, 6818, 58301, 132333] |
|6     |[3567, 51931, 7842, 60943, 59018] |
|7     |[6818, 40491, 8477, 148881, 5222] |
|8     |[3567, 6201, 4495, 7815, 174053]  |
|9     |[6818, 40491, 8477, 148881, 58301]|
|10    |[90888, 69069, 7842, 3404, 51931] |
+------+----------------------------------+
only showing top 10 rows



In [45]:
movieRecomments.select("movieId", "recommendations.userId").show(10, False)

+-------+------------------------+
|movieId|userId                  |
+-------+------------------------+
|1      |[53, 360, 543, 99, 12]  |
|3      |[543, 43, 243, 544, 53] |
|5      |[543, 43, 327, 544, 554]|
|6      |[53, 360, 276, 236, 99] |
|9      |[543, 327, 53, 243, 43] |
|12     |[543, 327, 243, 337, 43]|
|13     |[360, 53, 99, 12, 236]  |
|15     |[543, 327, 243, 43, 544]|
|16     |[53, 276, 258, 243, 171]|
|17     |[543, 35, 224, 90, 546] |
+-------+------------------------+
only showing top 10 rows



In [46]:
users = df_ratingsclean.select("userId").distinct().limit(3)
users.show()

+------+
|userId|
+------+
|   148|
|   540|
|    31|
+------+



In [47]:
userSubsetRecs = model.recommendForUserSubset(users, 10)

In [48]:
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   540|[{62293, 5.716651...|
|    31|[{3567, 6.165907}...|
|   148|[{62293, 5.116056...|
+------+--------------------+



In [49]:
userSubsetRecs.select("userId", "recommendations.movieId").show(10, False)

+------+----------------------------------------------------------------------+
|userId|movieId                                                               |
+------+----------------------------------------------------------------------+
|540   |[62293, 60943, 59018, 5075, 51931, 7842, 33649, 84847, 93988, 179135] |
|31    |[3567, 7815, 6201, 4495, 51931, 174053, 2563, 60943, 59018, 8235]     |
|148   |[62293, 60943, 59018, 33649, 5075, 84847, 93988, 6818, 179135, 138966]|
+------+----------------------------------------------------------------------+



In [50]:
movies = df_ratingsclean.select("movieId").distinct().limit(3)
movies.show()

+-------+
|movieId|
+-------+
|   1580|
|   2366|
|   3175|
+-------+



In [51]:
# Tạo ra 10 gợi ý xếp hạng người dùng cho tập hợp con sản phẩm đã được chỉ định (movies) dựa trên mô hình đã huấn luyện
movieSubSetRecs = model.recommendForItemSubset(movies, 10)
# Chọn các cột "movieId" và "recommendations.userId" từ DataFrame 'movieSubSetRecs'
# Hiển thị 10 dòng đầu tiên của kết quả, trong đó "recommendations.userId" là danh sách các người dùng được gợi ý cho từng sản phẩm
movieSubSetRecs.select("movieId", "recommendations.userId").show(10, False)

+-------+------------------------------------------------+
|movieId|userId                                          |
+-------+------------------------------------------------+
|1580   |[53, 543, 276, 452, 12, 371, 327, 578, 93, 25]  |
|3175   |[53, 276, 371, 543, 452, 171, 93, 99, 122, 1]   |
|2366   |[360, 138, 236, 20, 548, 99, 505, 191, 597, 251]|
+-------+------------------------------------------------+



In [52]:
# Tạo danh sách movie_ids và user_ids
movie_ids = [1580, 3175, 2366, 1590]
user_ids = [543, 543, 543, 543]

# Tạo DataFrame mới 'new_user_preds' từ danh sách movie_ids và user_ids
new_user_preds = sqlContext.createDataFrame(zip(movie_ids, user_ids), schema=["movieId", "userId"])

In [54]:
# Sử dụng mô hình để tạo ra các dự đoán xếp hạng cho DataFrame 'new_user_preds'
new_predictions = model.transform(new_user_preds)

# Hiển thị kết quả
new_predictions.show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|   1580|   543| 4.6188784|
|   3175|   543| 4.5366387|
|   2366|   543| 3.9088342|
|   1590|   543| 2.5014122|
+-------+------+----------+

